In [396]:
import pandas as pd
from pymongo import MongoClient
import re
import csv


### Connection avec mongodb


In [397]:
client = MongoClient("localhost:27017")
database = client.scrapy
movies = database.movies
serie = database.serie
movies.drop()
serie.drop()

### Nettoyage des données

In [398]:
df_movie = pd.read_csv("csv/movies.csv")
df_serie = pd.read_csv("csv/serie.csv")

In [399]:
df_movie["titre_original"].replace(to_replace="Original title: ", value='', regex=True,inplace=True)


In [400]:
# df_movie["duree"].replace(to_replace=",", value='', regex=True,inplace=True)
# df_serie["duree"].replace(to_replace=",", value='', regex=True,inplace=True)
# def convertion_duree(s):
    
#     split_cell = re.split("\s", s)
#     duree = 0
#     for elt in split_cell:
#         if "h" in elt:
#             duree = int(re.split("h", elt)[0]) *60 +duree
#         elif "m" in elt:
#             duree =int(re.split("m", elt)[0]) +duree
#     return duree
    




# df_movie['duree'] = df_movie['duree'].apply(lambda x :  convertion_duree(str(x)))
# df_serie['duree'] = df_serie['duree'].apply(lambda x :  convertion_duree(str(x)))



In [401]:
df_movie.to_csv("csv/movies.csv",index=False)
df_serie.to_csv("csv/serie.csv",index=False)

### Import des données dans mongodb

In [402]:
header = ["acteurs","date","descriptions","duree","genre","pays","public","score","titre","titre_original"]
csvfile = open('csv/movies.csv', 'r')
reader = csv.DictReader( csvfile )

for each in reader:
    row={}
    for field in header:

        if field in  ["acteurs","genre","pays"]:
            row[field]=each[field].split(",")
        elif field in ["date","duree"]:
            row[field]=int(each[field])
        elif field =="score":
            row[field]=float(each[field])
        else :
            row[field]=each[field]
        
    movies.insert_one(row)

In [403]:
header = ["acteurs","date","descriptions","duree","genre","pays","public","score","titre","titre_original"]
csvfile = open('csv/serie.csv', 'r')
reader = csv.DictReader( csvfile )

for each in reader:
    row={}
    for field in header:

        if field in  ["acteurs","genre","pays"]:
            row[field]=each[field].split(",")
        elif field in ["duree"]:
            row[field]=int(each[field])
        elif field =="score":
            row[field]=float(each[field])
        else :
            row[field]=each[field]  
        
    serie.insert_one(row)

## question n°1

In [404]:
movies.find().sort("duree",-1).limit(1)[0]

{'_id': ObjectId('6399f13472035cec05d071d8'),
 'acteurs': ['Clark Gable', 'Vivien Leigh', 'Thomas Mitchell'],
 'date': 1939,
 'descriptions': 'The manipulative daughter of a Georgia plantation owner conducts a turbulent romance with a roguish profiteer during the American Civil War and Reconstruction periods.',
 'duree': 238,
 'genre': ['Drama', 'Romance', 'War'],
 'pays': ['United States'],
 'public': 'Passed',
 'score': 8.2,
 'titre': 'Gone with the Wind',
 'titre_original': ''}

## Question n°2

In [405]:
five_movies=movies.find().sort("score",-1).limit(5)
for movie in five_movies:
    print(movie)

{'_id': ObjectId('6399f13472035cec05d0716c'), 'acteurs': ['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'], 'date': 1994, 'descriptions': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.', 'duree': 142, 'genre': ['Drama'], 'pays': ['United States'], 'public': 'R', 'score': 9.3, 'titre': 'The Shawshank Redemption', 'titre_original': ''}
{'_id': ObjectId('6399f13472035cec05d07264'), 'acteurs': ['Marlon Brando', 'Al Pacino', 'James Caan'], 'date': 1972, 'descriptions': 'The aging patriarch of an organized crime dynasty in postwar New York City transfers control of his clandestine empire to his reluctant youngest son.', 'duree': 175, 'genre': ['Crime', 'Drama'], 'pays': ['United States'], 'public': 'R', 'score': 9.2, 'titre': 'The Godfather', 'titre_original': ''}
{'_id': ObjectId('6399f13472035cec05d07261'), 'acteurs': ['Elijah Wood', 'Viggo Mortensen', 'Ian McKellen'], 'date': 2003, 'descriptions': "Gandalf and Aragorn 

## Question n°3


In [406]:
Freeman_movies = movies.find({"acteurs":{"$in":["Morgan Freeman"]}})
for movie in Freeman_movies:
    print(movie)

{'_id': ObjectId('6399f13472035cec05d0716c'), 'acteurs': ['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'], 'date': 1994, 'descriptions': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.', 'duree': 142, 'genre': ['Drama'], 'pays': ['United States'], 'public': 'R', 'score': 9.3, 'titre': 'The Shawshank Redemption', 'titre_original': ''}
{'_id': ObjectId('6399f13472035cec05d071c9'), 'acteurs': ['Hilary Swank', 'Clint Eastwood', 'Morgan Freeman'], 'date': 2004, 'descriptions': 'A determined woman works with a hardened boxing trainer to become a professional.', 'duree': 132, 'genre': ['Drama', 'Sport'], 'pays': ['United States'], 'public': 'PG-13', 'score': 8.1, 'titre': 'Million Dollar Baby', 'titre_original': ''}
{'_id': ObjectId('6399f13472035cec05d071ec'), 'acteurs': ['Clint Eastwood', 'Gene Hackman', 'Morgan Freeman'], 'date': 1992, 'descriptions': 'Retired Old West gunslinger William Munny reluctantly takes on one l

In [407]:
Cruise_movies = movies.find({"acteurs":{"$in":["Tom Cruise"]}})
for movie in Cruise_movies:
    print(movie)

{'_id': ObjectId('6399f13472035cec05d07224'), 'acteurs': ['Tom Cruise', 'Jennifer Connelly', 'Miles Teller'], 'date': 2022, 'descriptions': "After thirty years, Maverick is still pushing the envelope as a top naval aviator, but must confront ghosts of his past when he leads TOP GUN's elite graduates on a mission that demands the ultimate sacrifice from those chosen to fly it.", 'duree': 130, 'genre': ['Action', 'Drama'], 'pays': ['United States'], 'public': 'PG-13', 'score': 8.4, 'titre': 'Top Gun: Maverick', 'titre_original': ''}


## Question n°4

In [408]:
horror_movies = movies.find({"genre":{"$in":["Horror"]}}).sort("score",-1).limit(3)
for movie in horror_movies:
    print(movie)

{'_id': ObjectId('6399f13472035cec05d0724b'), 'acteurs': ['Anthony Perkins', 'Janet Leigh', 'Vera Miles'], 'date': 1960, 'descriptions': "A Phoenix secretary embezzles $40,000 from her employer's client, goes on the run and checks into a remote motel run by a young man under the domination of his mother.", 'duree': 109, 'genre': ['Horror', 'Mystery', 'Thriller'], 'pays': ['United States'], 'public': 'R', 'score': 8.5, 'titre': 'Psycho', 'titre_original': ''}
{'_id': ObjectId('6399f13472035cec05d0723f'), 'acteurs': ['Sigourney Weaver', 'Tom Skerritt', 'John Hurt'], 'date': 1979, 'descriptions': 'The crew of a commercial spacecraft encounter a deadly lifeform after investigating an unknown transmission.', 'duree': 117, 'genre': ['Horror', 'Sci-Fi'], 'pays': ['United Kingdom', 'United States'], 'public': 'R', 'score': 8.5, 'titre': 'Alien', 'titre_original': ''}
{'_id': ObjectId('6399f13472035cec05d07236'), 'acteurs': ['Jack Nicholson', 'Shelley Duvall', 'Danny Lloyd'], 'date': 1980, 'des

In [409]:
drama_movies = movies.find({"genre":{"$in":["Drama"]}}).sort("score",-1).limit(3)
for movie in drama_movies:
    print(movie)

{'_id': ObjectId('6399f13472035cec05d0716c'), 'acteurs': ['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'], 'date': 1994, 'descriptions': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.', 'duree': 142, 'genre': ['Drama'], 'pays': ['United States'], 'public': 'R', 'score': 9.3, 'titre': 'The Shawshank Redemption', 'titre_original': ''}
{'_id': ObjectId('6399f13472035cec05d07264'), 'acteurs': ['Marlon Brando', 'Al Pacino', 'James Caan'], 'date': 1972, 'descriptions': 'The aging patriarch of an organized crime dynasty in postwar New York City transfers control of his clandestine empire to his reluctant youngest son.', 'duree': 175, 'genre': ['Crime', 'Drama'], 'pays': ['United States'], 'public': 'R', 'score': 9.2, 'titre': 'The Godfather', 'titre_original': ''}
{'_id': ObjectId('6399f13472035cec05d07261'), 'acteurs': ['Elijah Wood', 'Viggo Mortensen', 'Ian McKellen'], 'date': 2003, 'descriptions': "Gandalf and Aragorn 

In [410]:
comedy_movies = movies.find({"genre":{"$in":["Comedy"]}}).sort("score",-1).limit(3)
for movie in comedy_movies:
    print(movie)

{'_id': ObjectId('6399f13472035cec05d0724d'), 'acteurs': ['Roberto Benigni', 'Nicoletta Braschi', 'Giorgio Cantarini'], 'date': 1997, 'descriptions': 'When an open-minded Jewish waiter and his son become victims of the Holocaust, he uses a perfect mixture of will, humor, and imagination to protect his son from the dangers around their camp.', 'duree': 116, 'genre': ['Comedy', 'Drama', 'Romance'], 'pays': ['Italy'], 'public': 'PG-13', 'score': 8.6, 'titre': 'Life Is Beautiful', 'titre_original': 'La vita è bella'}
{'_id': ObjectId('6399f13472035cec05d07242'), 'acteurs': ['Charles Chaplin', 'Virginia Cherrill', 'Florence Lee'], 'date': 1931, 'descriptions': '', 'duree': 87, 'genre': ['Comedy', 'Drama', 'Romance'], 'pays': ['United States'], 'public': 'G', 'score': 8.5, 'titre': 'City Lights', 'titre_original': ''}
{'_id': ObjectId('6399f13472035cec05d0716f'), 'acteurs': ['François Cluzet', 'Omar Sy', 'Anne Le Ny'], 'date': 2011, 'descriptions': 'After he becomes a quadriplegic from a par

## Question 6

In [411]:
american_movies = movies.find().sort("score",-1).limit(100)
count=0
for movie in american_movies:
    if "United States" in movie["pays"]:
        count +=1
pourcentage = count/250
print(pourcentage*100)

29.2


In [412]:
french_movies = movies.find({"pays":{"$in":["France"]}}).sort("score",-1).limit(100)
for movie in french_movies:
    print(movie["titre"],movie["score"])

City of God 8.6
The Intouchables 8.5
Léon: The Professional 8.5
Cinema Paradiso 8.5
The Pianist 8.5
Capernaum 8.4
The Boat 8.4
Amadeus 8.4
The Lives of Others 8.4
A Separation 8.3
Incendies 8.3
Amélie 8.3
The Passion of Joan of Arc 8.2
The Wages of Fear 8.2
Ran 8.2
Casino 8.2
The Father 8.2
The 400 Blows 8.1
La haine 8.1
Before Sunset 8.1
Wild Tales 8.1


In [419]:
top_100 = movies.find().sort("score", -1)[:100]
france = 0
us = 0
for i in top_100:
    if "France" in i['pays']  :
        france += 1
    elif "United States" in i['pays'] :
        us += 1
f"Dans le Top 100 des films : US ({us}%) FR ({france}%)"

'Dans le Top 100 des films : US (71%) FR (12%)'

## Question n°7


In [414]:
liste_genre =[]
list_movies = movies.find()
for movie in list_movies:
    genre_movie = movie["genre"]
    for genre in genre_movie:
        if not genre in liste_genre:
            liste_genre.append(genre)
print(liste_genre, len(liste_genre))

['Drama', 'Romance', 'War', 'Action', 'Mystery', 'Biography', 'Comedy', 'Western', 'Animation', 'Music', 'Sci-Fi', 'Crime', 'Adventure', 'History', 'Thriller', 'Family', 'Film-Noir', 'Fantasy', 'Horror', 'Sport', 'Musical'] 21


In [415]:
dict_temps_genre={}
for genre in liste_genre:
    temps_total = 0
    list_movies = movies.find({"genre":{"$in":[genre]}})
    nb_movies=list_movies.count()
    
    for movie in list_movies:
        temps_total = movie["duree"] + temps_total

    dict_temps_genre[genre] =  temps_total/nb_movies



/tmp/ipykernel_4245/386818461.py:5: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  nb_movies=list_movies.count()


In [416]:
for key,value in dict_temps_genre.items():
    print(f"Le temps moyen du genre {key} est {round(value,2)} minutes ")

Le temps moyen du genre Drama est 134.64 minutes 
Le temps moyen du genre Romance est 118.48 minutes 
Le temps moyen du genre War est 140.43 minutes 
Le temps moyen du genre Action est 133.92 minutes 
Le temps moyen du genre Mystery est 124.61 minutes 
Le temps moyen du genre Biography est 147.07 minutes 
Le temps moyen du genre Comedy est 106.33 minutes 
Le temps moyen du genre Western est 149.33 minutes 
Le temps moyen du genre Animation est 100.96 minutes 
Le temps moyen du genre Music est 134.25 minutes 
Le temps moyen du genre Sci-Fi est 131.7 minutes 
Le temps moyen du genre Crime est 131.78 minutes 
Le temps moyen du genre Adventure est 129.42 minutes 
Le temps moyen du genre History est 156.0 minutes 
Le temps moyen du genre Thriller est 121.03 minutes 
Le temps moyen du genre Family est 114.54 minutes 
Le temps moyen du genre Film-Noir est 110.0 minutes 
Le temps moyen du genre Fantasy est 123.57 minutes 
Le temps moyen du genre Horror est 120.6 minutes 
Le temps moyen du genr